This first cell loads the packages that we will be using.

In [1]:
using DrWatson
@quickactivate "Doran_etal_2022"

using SPI
using Muon
using CSV, DataFrames

For the purposes of this demo we will use a dataset of 630 biobank strains. Ideally, of course we would be working with a larger and more diverse dataset, such as the 7047 non-redundent reference proteomes.

In [2]:
uniprot = readh5ad(joinpath(datadir(), "exp_raw", "UP7047", "2022-02-22_UP7047.h5ad"))
mtx = uniprot.X[:,:]; # load ogg count matrix into RAM memory
# bmtx = Float64.(mtx .> 0); # ignore abundence

In [3]:
@time usv = svd(mtx); # takes about 4-5 mins 

255.418351 seconds (332.30 k allocations: 2.567 GiB, 0.13% gc time, 0.05% compilation time)


In [4]:
uniprot.obsm["LSVs"] = usv.U
uniprot.varm["RSVs"] = usv.V
uniprot.uns["SVs"] = usv.S;

In [5]:
usv1 = SVD(uniprot.obsm["LSVs"][:,:], uniprot.uns["SVs"][:], uniprot.varm["RSVs"][:,:]');

Here we check that our svd decomposition does in fact match the original matrix we factorized.

i.e. we are checking $USV - M \le \epsilon $ 

In [6]:
projectout(usv1) ≈ mtx

true

we can also give projectout() a window to use `projectout(usv1, 1:10)` computes USV on first 10 PCs.

## Calculating the SPI distance matrix

The next step is to compute the SPI distance matrices. There are a few ways of calling this function.

Remember that SPI relies on two conceptual parts, how we partition the spectum and how we compute the distance within that partition.

Thus there are two main functions in SPI

```julia
getintervals(singularvalues)
```

and 

```julia
calc_spi_mtx()
```

This is the entirety of `calc_spi_mtx()`

```julia
function calc_spi_mtx(vecs::AbstractMatrix{<:T}, vals::AbstractVector{<:T}, intervals::AbstractVector) where T<:Number
    sprmtx = zeros(size(vecs,1), size(vecs,1))
    for grp in intervals
        sprmtx += Distances.pairwise(WeightedEuclidean(vals[grp]), vecs'[grp,:])
    end
    return sprmtx.^2
end
```

There are a few different ways to call the `calc_spi_mtx()` function.

We could just provide the full SVD object
`dij = calc_spi_mtx(usv)`
by default this will compute the rowwise distances.

We could specify that we want the colwise distances by providing the full SVD object and an option to dispatch on the `V` matrix

```julia
dij = calc_spi_mtx(usv, SPI.RSVs) # will compute the colwise distances
```

Alternatively we can explicitly provide the vectors and values to the function. Along with a vector of Ranges that specify how to split the spectrum. 

If we wanted to not split the spectrum at all, the call would look like

`calc_spi_mtx(usv.U, usv.S, [1:length(usv.S)])`

Or if we wanted each component treated individually we could call

`calc_spi_mtx(usv.U, usv.S, [n:n for n in 1:length(usv.S)])`

But, for now we should stick with using the `getintervals()` function. 

This is the entirety of the `getintervals()`

```julia
function getintervals(S::AbstractVector{<:Number}; alpha=1.5, q=.75)
    potentialbreaks = abs.(diff(log.(S.+1)))
    θ = alpha * quantile(potentialbreaks, q)
    breaks = findall(potentialbreaks .> θ) .+ 1
    starts, ends = vcat(1, breaks), vcat(breaks.-1, length(S))
    intervals = map((s,e)->s:e, starts, ends)
    return intervals
end
```

This function finds spectral partitions by computing the log difference between each subsequent singular
value and by default selects the differences that are larger than `1.5 * Q3(differences)`

i.e. finds breaks in the spectrum that explain far smaller scales of variance

Args:
* S = singular values of a SVD factorization
* alpha = scalar multiple of `q`
* q = which quantile of log differences to use; by default Q3 

Returns:
* AbstractVector{UnitRange} indices into S corresponding to the spectral partitions

In [7]:
@time obs_dij = calc_spi_mtx(usv.U, usv.S, getintervals(usv.S)); # takes ~6min
obs_dij ./ size(mtx, 2) # converts from appox num of "mutations" to "mutations/nfeatures"
uniprot.obsp["SPI_distances"] = obs_dij;

436.517513 seconds (4.50 M allocations: 652.949 GiB, 10.54% gc time, 0.22% compilation time)


In [8]:
@time var_dij = calc_spi_mtx(usv.V, usv.S, getintervals(usv.S)); # takes ~20min
uniprot.varp["SPI_distances"] = var_dij;

1286.527464 seconds (83.82 k allocations: 1.329 TiB, 6.38% gc time, 0.01% compilation time)


## We next will want to calculate a tree from the SPI distance matrix. 

So far I have been using basic hierarchical clustering functions.

To connect hierarchical clustering to the field of phylogeny estimation. 

hclust with average linkage is equal to UPGMA (unweighted pair group method with arithmetic mean) tree forming

Unfortunately this method has some bad assumptions about a constant rate of evolution, 
but is very simple to compute, and has worked so far on the UniProt and BioBank datasets I have been working with.

https://en.wikipedia.org/wiki/UPGMA

We can calculate the tree as so...

In [9]:
obstree = hclust(obs_dij, linkage=:average, branchorder=:optimal);

In [10]:
fieldnames(typeof(obstree))

(:merges, :heights, :order, :linkage)

In [11]:
@time nwtreestring = nwstr(obstree, uniprot.obs_names.vals; labelinternalnodes=false)

  0.621247 seconds (1.44 M allocations: 362.403 MiB, 7.94% gc time, 85.19% compilation time)


"((((UP000023198:8.779049e+04,((UP000244959:8.151764e+04,((UP000020681:7.867282e+04,(((UP000006735:7.526238e+04,((UP000214646:7.438872e+04,(UP000004508:7.385921e+04,(((UP000000671:7.100427e+04,((UP000186474:5.273874e+04,UP000004095:5.273874e+04):1.797993e+04,(((UP000037" ⋯ 281327 bytes ⋯ "7e+03,UP000185464:8.087722e+04):6.404236e+02):1.377056e+03,UP000064967:8.289470e+04):4.895798e+03):7.908267e+02,(UP000189670:7.776935e+04,UP000037988:7.776935e+04):1.081197e+04):1.897709e+03,UP000293874:9.047903e+04):6.698245e+03,UP000003922:9.717728e+04):0.000000e+00;"

In [12]:
uniprot.uns["obs_spitree_newickstring"] = nwtreestring;

In [13]:
uniprot.obs[:,"order"] = obstree.order
uniprot.uns["obsmerges"] = obstree.merges
uniprot.uns["heights"] = obstree.heights
uniprot.uns["treelinkage"] = string(obstree.linkage)

"average"

In [14]:
mkpath(joinpath(datadir(), "exp_pro", "UP7047"))
writeh5ad(joinpath(datadir(), "exp_pro", "UP7047", "2022-02-22_UP7047.h5ad"), uniprot)


So far we have compute the basic SPI tree, we may also want some measure of how statistically confident we are on each of the branches and merges we have predicted in the tree. That is best performed through a boot strap analysis. Where we sample with replacement the features and recompute the SPI distance matrix and tree.

For a bootstrap analysis run this script:

```bash
julia -p 5 scripts/computebootstraps.jl \
           -i data/exp_raw/UP7047/2022-02-22_UP7047.h5ad \
           -o _research/runSPIonUP7047rows \
           -b 100
```

In [15]:
# cd(projectdir())
# run(`julia -p 5 scripts/computebootstraps.jl \
#     -i data/exp_raw/UP7047/2022-02-22_UP7047.h5ad \
#     -o _research/runSPIonUP7047rows \
#     -n 5`)

In [ ]:
using Random: sample 
using Gotree_jll

In [33]:
Threads.nthreads()

1

In [ ]:
nboot = 5
boottrees = Vector{String}(undef, nboot)
Threads.@threads for i in 1:nboot
    nfeats = size(bmtx, 2)
    colsmps = sample(1:nfeats, nfeats, replace=true)
    tmpM = bmtx[:,colsmps]
    vals, vecs = eigen(Matrix(tmpM * tmpM'))
    dij = calc_spi_mtx(vecs, sqrt.(max.(vals, zero(eltype(vals)))))
    dij = dij ./ nfeats
    hc = hclust(dij, linkage=:average, branchorder=:optimal)
    boottrees[i] = nwstr(hc, uniprot.obs_names.vals; labelinternalnodes=false)
end

In [ ]:
## write out SPI trees
outputdir = mkpath(joinpath(datadir, "exp_pro", "uniprot_spitrees"))
open(joinpath(outputdir, "uniprot-tree.nw"), "w") do io
    println(io, nwtreestring)
end
open(joinpath(outputdir, "uniprot-boottrees.nw"), "w") do io
    for btree in boottrees
        println(io, btree)
    end
end

In [ ]:

## calculate support
run(pipeline(`$(gotree()) compute support tbe --silent \
    -i $(joinpath(outputdir, "uniprot-tree.nw")) \
    -b $(joinpath(outputdir, "uniprot-boottrees.nw")) \
    -o $(joinpath(outputdir, "uniprot-supporttree.nw"))`,
    stderr=joinpath(outputdir, "booster.log")))